# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [29]:
import pandas as pd

url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
customer_data = pd.read_csv(url)

def clean_data(df: pd.DataFrame):
    df = df.rename(columns={"AT": "state"})
    df = df.rename(columns={"State": "st"})
    df = df.rename(columns={"state": "st"})
    df.columns = pd.Series(df.columns).apply(lambda col: col.replace(" ", "_").lower())
    df['monthly_premium_auto'] = df['monthly_premium_auto'].astype(float)    
    df['income'] = df['income'].astype(float)      
    df['total_claim_amount'] = df['total_claim_amount'].astype(float)            

    gender_clean = {
        "Femal": "F",
        "female": "F",
        "Male": "M",
        "MALE": "M",
        "FEMALE": "F" 
    }

    df['gender'] = df['gender'].str.upper().replace(gender_clean)

    st_clean = {
        "Cali": "California",
        "AZ": "Arizona",
        "Az": "Arizona",
        "WA": "Washington"
    }

    df['st'] = df['st'].str.lower().replace(st_clean)

    ed_clean = {"Bachelors": "Bachelor", "College": "Bachelor"}
    df['education'] = df['education'].str.lower().replace(ed_clean)

    # Clean customer_lifetime_value
    if not pd.api.types.is_float_dtype(df['customer_lifetime_value']):
        df['customer_lifetime_value'] = df['customer_lifetime_value'].str.replace("%", "").astype(float)

    vh_clean = {
        "Sports Car": "Luxury",
        "Luxury SUV": "Luxury",
        "Luxury Suv": "Luxury",
        "Luxury Car": "Luxury"
    }

    df['vehicle_class'] = df['vehicle_class'].str.lower().replace(vh_clean)
    
    # Fill NaNs and other cleaning
    if len(df['number_of_open_complaints']):
        df['number_of_open_complaints'] = df['number_of_open_complaints'].fillna("0/0/00")
        df['number_of_open_complaints'] = df['number_of_open_complaints'].apply(lambda x: int(x[2]) if isinstance(x, str) and len(x) > 1 else 0)

    fill_values = {
        'customer': "AA00000",
        'st': "Other",
        'gender': "Unknown",
        'education': "Unknown",
        'income': "Unknown",
        'monthly_premium_auto': "Unknown",
        'policy_type': "Unknown",
        'vehicle_class': "Unknown",
        'total_claim_amount': "Unknown"
    }

    for column, value in fill_values.items():
        df[column] = df[column].fillna(value)

    df['customer_lifetime_value'] = df['customer_lifetime_value'].fillna(df['customer_lifetime_value'].mean())
    
    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)

    return df

customer_data_clean = clean_data(customer_data)
customer_data_clean = customer_data_clean.drop(columns=['unnamed:_0'])

display(customer_data_clean["response"].unique())


array(['No', 'Yes', nan], dtype=object)

In [2]:
"""
Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and 
have a response of "Yes" to the last marketing campaign.
"""

customer_data_clean1 = customer_data_clean[customer_data_clean['total_claim_amount'] > 1000]
customer_data_clean1.groupby('response').get_group('Yes').head()



,customer,st,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
189,OK31456,california,11009.130490,Yes,Premium,bachelor,1/24/11,Employed,F,51643.0,...,0,1,Corporate Auto,Corporate L3,Offer2,Agent,1358.400000,luxury car,Medsize,NaN
236,YJ16163,oregon,11009.130490,Yes,Premium,bachelor,1/24/11,Employed,F,51643.0,...,0,1,Special Auto,Special L3,Offer2,Agent,1358.400000,luxury car,Medsize,A
419,GW43195,oregon,25807.063000,Yes,Extended,college,2/13/11,Employed,F,71210.0,...,0,2,Personal Auto,Personal L2,Offer1,Branch,1027.200000,luxury car,Small,A
442,IP94270,arizona,13736.132500,Yes,Premium,master,2/13/11,Disabled,F,16181.0,...,0,8,Personal Auto,Personal L2,Offer1,Web,1261.319869,suv,Medsize,A
587,FJ28407,california,5619.689084,Yes,Premium,high school or below,1/26/11,Unemployed,M,0.0,...,0,1,Personal Auto,Personal L1,Offer2,Web,1027.000029,suv,Medsize,A


In [5]:
"""
Using the original Dataframe, analyze the average total_claim_amount by each policy type and 
gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.
"""

customer_data_clean2 = customer_data_clean.groupby('response').get_group('Yes')
display(customer_data_clean2.pivot_table(index='policy_type', columns='gender', values='total_claim_amount', aggfunc='mean').round(2))

#Conclusions: for personal auto policies, the average claim amount was similar for women and men. The average claim amount for men was slightly higher but not relevantly so. 
#In the special and corporate auto categories, women had higher average claim amounts.

gender,F,M
policy_type,,
Corporate Auto,433.74,408.58
Personal Auto,452.97,457.01
Special Auto,453.28,429.53


In [19]:
"""
Analyze the total number of customers who have policies in each state, and then filter the results to 
only include states where there are more than 500 customers.
"""

display(customer_data_clean.groupby('st').size())

#all states have over 500 customers, no filtering required

st
Other          631
arizona       1937
california    3552
nevada         993
oregon        2909
washington     888
dtype: int64

In [35]:
"""
Find the maximum, minimum, and median customer lifetime value by education level and gender. 
Write your conclusions.
"""

customer_data_clean.groupby(['education', 'gender'])['customer_lifetime_value'].agg(['max', 'min', 'median']).round(2)

#Conclusions: Males with a high school level of education or below have significantly higher maximum and median customer lifetime values than any other category. 

max      min   median
education            gender                            
bachelor             F       73225.96  1904.00  5640.51
                     M       67907.27  1898.01  5548.03
college              F       61850.19  1898.68  5623.61
                     M       61134.68  1918.12  6005.85
doctor               F       44856.11  2395.57  5332.46
                     M       32677.34  2267.60  5577.67
high school or below F       55277.45  2144.92  6039.55
                     M       83325.38  1940.98  6286.73
master               F       51016.07  2417.78  5729.86
                     M       50568.26  2272.31  5579.10

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here